In [1]:
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer
from PyPDF2 import PdfReader
from transformers import pipeline

import chromadb
import torch
import ollama

/home/nayra/.local/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
embeddings = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")

In [3]:
PATH = "pdf/JavaBasico.pdf"


reader = PdfReader(PATH)

text_load = ''.join(page.extract_text() for page in reader.pages)

client = chromadb.PersistentClient(path="chromadb")
collection = client.get_or_create_collection(name="vector-langchain")
history_collection = client.get_or_create_collection(name="chat_history")

In [4]:
text_split = []
metadata=[]
chunk_size = 1024
chunk_overlap = 8

for i in range(0, len(text_load), chunk_size):
    start = i
    end = i + chunk_size
    if start != 0:
        start = start - chunk_overlap
        end =  end + chunk_overlap
    text_split.append(text_load[start:end])
    metadata.append({"name": "book-langchain", "partition": f"{i}"})
embeddings_db = embeddings.encode(text_split)

In [5]:
for i, d in enumerate(text_split):

      collection.add(
              ids=[str(i)],
              embeddings=embeddings_db[[i]],
              documents=[d],
              metadatas=metadata[i]
    )

In [17]:
while True:

    chat_history = []
    prompt = input("Pergunte algo")
    response = embeddings.encode([prompt])
    results = collection.query(
            query_embeddings=response,
            n_results=2,
            where={"name": "book-langchain"}
        )

    data = results['documents'][0]
    distance = results['distances']
    
    for k in range(2):
    
        if distance[0][k] > 0.9:
            data = "Você é um assistente de chatbot e tem acesso limitado a informações. Nesse caso, diga que não tem conhecimento acerca da pergunta."
    
    output = ollama.generate(
            model="llama2",
            prompt=f"Você é um chatbot assistente. Com base no histórico de conversa: {chat_history}. Responda em português baseado somente na seguinte informação: {data}"
            )
    
    output = output['response']
    print(output)

    chat_history.append(f"Human: {prompt}\nAi: {output}")
    # embeddings_history = embeddings.encode(chat_history)

    # history_collection.add(
    #             ids=[str(i)],
    #             embeddings=embeddings_history[[i]],
    #             documents=[chat_history]
    # )

Pergunte algo o que é python?



Entendi! Sem mais informações disponíveis, peço desculpas, mas não posso fornecer respostas precisas ou úteis neste momento. Como assistente de chatbot, tenho acesso limitado a informações e não posso garantir a precisão ou validade das informações fornecidas. Se você tiver alguma outra pergunta ou precisar de ajuda em algum assunto, estou á disposição para ajudá-lo.


Pergunte algo o que é classe em orientação a objeto?


Bem-vindo! Em relação às confusões sobre classes e objetos, é importante entender que uma classe representa um modelo ou protótipo de um objeto, enquanto os objetos são instâncias concretas dos modelos.

Uma classe pode ser pensada como um plano de construção para um edifício, onde você tem todos os detalhes do projeto, desde a forma do prédio até as características de cada quarto. Já os objetos são as instâncias concretas desse edifício, com suas próprias características únicas.

No mundo real, você teria uma classe "Carro", que representa o modelo de um carro. Cada carro é um objeto instanciado da classe "Carro", com características específicas, como cor, modelo, combustível, etc.

Nosso sistema de negócios utiliza a mesma abordagem. Temos classes que representam os objetos de negócio, e cada objeto é uma instância concreta dessa classe, com características únicas. Por exemplo, temos uma classe "Aluno", que representa o modelo de um aluno do sistema. Cada aluno é um objeto instanciad

KeyboardInterrupt: Interrupted by user